In [14]:
import cv2
import torch
from pathlib import Path
from tqdm import tqdm


In [18]:
import torch.nn as nn

In [ ]:
from yolov5.models.experimental import attempt_load 
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression, scale_coords
from yolov5.utils.plots import plot_one_box
import os

# Set the device
device = torch.device("cpu")

# Load YOLOv5 model
model = attempt_load("yolov5s.pt", map_location=device)

# Set confidence threshold
conf_threshold = 0.3

# Define input and output paths
inputPath = os.getcwd() + "/test_images/"
outputPath = os.getcwd() + "/output_images/"

# Create output directory if it doesn't exist
Path(outputPath).mkdir(parents=True, exist_ok=True)

def detectVehicles(filename):
    global model, inputPath, outputPath, conf_threshold

    img = cv2.imread(inputPath + filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Inference
    results = model(img, size=640)

    # Post-processing
    results = non_max_suppression(results, conf_threshold, 0.4)

    for i, det in enumerate(results):
        if det is not None and len(det):
            # Rescale boxes from 640 to original image size
            det[:, :4] = scale_coords(img.shape[:2], det[:, :4], img.shape).round()

            for *xyxy, conf, cls in reversed(det):
                label = model.names[int(cls)]
                if label in ["car", "bus", "bike", "truck", "rickshaw"]:
                    plot_one_box(xyxy, img, label=label, color=(0, 255, 0), line_thickness=3)

    # Save output image
    outputFilename = outputPath + "output_" + filename
    cv2.imwrite(outputFilename, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    print('Output image stored at:', outputFilename)

# Loop through input images
for filename in os.listdir(inputPath):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        detectVehicles(filename)

print("Done!")
